In [2]:
from openai import AzureOpenAI
import pandas as pd
import json

### With Modules Installed we will define our AzureOpenAI connectivity

In [3]:
# Define connection parameters
AZURE_OPENAI_ENDPOINT = "AZURE_OPENAI_ENDPOINT"
AZURE_OPENAI_MODEL = "AZURE_OPENAI_MODEL"
AZURE_OPENAI_KEY = "AZURE_OPENAI_KEY"

In [4]:
# Lets create a test case for the model to validate the connection and Json output
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_KEY,  
  api_version="2023-12-01-preview"
)

response = client.chat.completions.create(
  model=AZURE_OPENAI_MODEL, # Model = should match the deployment name you chose for your 1106-preview model deployment
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

{
  "WorldSeriesWinner2020": {
    "TeamName": "Los Angeles Dodgers",
    "Defeated": "Tampa Bay Rays",
    "SeriesResult": "4-2",
    "Notable": "It was the Dodgers' seventh World Series title"
  }
}


Now let's use GitHub copilot to turn that example from our docs page which evaluates if the connection is working into a Python function that we can repeatable call over our data. 

In [5]:
def get_chat_completion_json(system_content, user_content):
    response = client.chat.completions.create(
        model=AZURE_OPENAI_MODEL,  # Model = should match the deployment name you chose for your 1106-preview model deployment
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ]
    )
    return json.loads(response.choices[0].message.content)

In [6]:
#Lets test the function
result = get_chat_completion_json("You are a helpful assistant designed to output JSON.", "Who won the world series in 2020?")
print(result)

{'World_Series_Winner_2020': {'Team': 'Los Angeles Dodgers', 'Defeated': 'Tampa Bay Rays', 'Series_Score': '4-2', 'First_Title_Since': '1988'}}


Execellent, now that we have a working function, lets pivot to look at our dataset for today. Using ChatGPT I've created 9 samples reviews for our fake BBQ product, the "Cooktop Portable BBQ". The objective will be to not just perform sentiment analysis, but also understand what next best action to take to improve on the feedback. Let's start by loading in the reviews from our txt file

In [7]:
# Read the text file line by line
with open('Reviews.txt', 'r') as file:
    lines = file.readlines()

# Remove any trailing new line characters
lines = [line.rstrip('\n') for line in lines]

# Convert the list into a DataFrame
df = pd.DataFrame(lines, columns=['reviews'])

# Display the DataFrame
print(df.head(5))

                                             reviews
0  Just bought the Cookup Portable BBQ from Outdo...
1  I picked up the Cookup Portable BBQ from Adven...
2  I'm thrilled with my Cookup Portable BBQ from ...
3  Bought the Cookup Portable BBQ from Grill Mast...
4  Purchased this BBQ from Patio World. It's ligh...


Now that we have the reviews loaded, we need to define a system message for GPT-4 understand the JSON format that we want. We'd like to extract the user sentiment and also a recommended next step we can take to improve the customer experience and product. Let's start with the following prompt.

In [8]:
SysMsg = """
Your are a helpful assistant designed to output JSON. You will be given a review for a product and you should 
output the following JSON format.
{
    "sentiment": "positive" or "negative" or "neutral",
    "recommended_store_action":"If the review is negative or neutral, what is an action we can take take to improve the customer experience in the retail store, if it positive review or you're unsure simply state not applicable",
    "recommended_product_action":"If the review is negative or neutral, what is an action we can take take to improve the product, if it positive review or you're unsure simply state not applicable"
}
"""

Now let's use this message with the first entry in our data frame

In [9]:
result = get_chat_completion_json(SysMsg, df['reviews'][0])
print(result)

{'sentiment': 'positive', 'recommended_store_action': 'not applicable', 'recommended_product_action': 'not applicable'}


Amazing! Now let's try it out on a negative review!

In [10]:
result = get_chat_completion_json(SysMsg, df['reviews'][8])
print(result)

{'sentiment': 'negative', 'recommended_store_action': 'Provide additional training to staff to improve customer service and responsiveness to customer concerns.', 'recommended_product_action': "Investigate the product's design to improve ease of setup and heating distribution. Consider enlarging the grilling space to accommodate users' needs."}


Alright, now let's apply this to every row in the dataframe! I'm unsure how to make the code, so we'll use GitHub Copilot!

In [13]:
def apply_get_chat_completion_json(review):
    # Replace 'system_content' with the actual system content you want to use
    system_content = SysMsg
    response = get_chat_completion_json(system_content, review)
    return pd.Series(response)

# Apply the function to the 'reviews' column
results_df = df['reviews'].apply(apply_get_chat_completion_json)

Now Let's look at the results of the process in the dataframe!

In [14]:
# Print the DataFrame to check the result
results_df.head(10)

,sentiment,recommended_store_action,recommended_product_action
0,positive,not applicable,not applicable
1,positive,not applicable,not applicable
2,positive,not applicable,not applicable
3,neutral,Train staff to engage more warmly with custome...,Consider designing a new model with a larger g...
4,neutral,Provide additional training to staff to ensure...,Investigate BBQ design for potential improveme...
5,neutral,Provide staff training to ensure proactive cus...,Investigate heat distribution issues and consi...
6,negative,Improve staff training on customer service to ...,Reassess the build quality and ignition system...
7,negative,Provide staff with additional training to impr...,Review the BBQ product design to improve sturd...
8,negative,Provide staff with customer service training t...,Review the BBQ design to improve ease of setup...


Now we have processed all the customer feedback, we can then pass the collected feedback back through the OpenAI service and perform and aggregate recommendation using the standard text in text out experience. 
Let's define another function to use, and then pass through all of our recommended product actions.

In [15]:
def get_chat_completion_text(system_content, user_content):
    response = client.chat.completions.create(
        model=AZURE_OPENAI_MODEL,  # Model = should match the deployment name you chose for your 1106-preview model deployment
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ]
    )
    return response.choices[0].message.content

In [16]:
# Select rows where 'recommended_product_action' is not "not applicable"
selected_rows = results_df.loc[results_df['recommended_product_action'] != "not applicable", 'recommended_product_action']

# Concatenate the selected items into a string with comma separating each item
product_summary = ', '.join(selected_rows)

# Print the output string
print(product_summary)

Consider designing a new model with a larger grilling area while maintaining portability to meet customer needs., Investigate BBQ design for potential improvements in heating efficiency and consider including clear instructions or tips for optimal temperature management., Investigate heat distribution issues and consider design improvements., Reassess the build quality and ignition system of the Portable BBQ to improve its durability and reliability., Review the BBQ product design to improve sturdiness and heat control consistency. Consider reaching out to the customer to offer a replacement or refund., Review the BBQ design to improve ease of setup and ensure even heating. Consider making a version with more grilling space to accommodate customers' needs.


Let's now pass these recommendations into the model, defining a relevant system message first. This will be much more token efficient than simply parsing the entire review, and hopefully more accurate. 

In [17]:
SysMsgReview = """
Your are a helpful assistant designed to help provide a recommendation for best path forward for product improvement.
Given many recommended actions for remmediation, you should consider which recommendations might be the easiest to
implement for our portable barbeque product, and which recommendations are the most requested.
"""

print(get_chat_completion_text(SysMsgReview, product_summary))

To determine the best path forward for product improvement for the portable barbecue product, we need to consider factors such as the impact on customer satisfaction, cost, implementation time, and feasibility. Looking at the suggestions:

1. **Design a new model with a larger grilling area while maintaining portability**: This could significantly increase customer satisfaction for those who require more space. However, redesigning and producing a new model could be time-consuming and expensive.

2. **Investigate BBQ design for potential improvements in heating efficiency and temperature management instructions or tips**: This may be easier to implement as it could involve tweaks to the current design rather than a full redesign. Providing better instructions or tips could be quite straightforward and cost-effective. 

3. **Investigate heat distribution issues and design improvements**: This is critical as even heating is a fundamental aspect of a good BBQ. Changes could range from min